In [2]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import  plotly as py
import pandas as pd

# Total population, N.
N = 1000000
# Initial number of infected and recovered individuals, I0 and R0.
IA0, IP0, IM0, IH0, IC0, R10, R20, D0 = 1, 1, 1, 1, 1, 0, 0, 0
E0 = np.arange(2, 500, 1)[0]
# Everyone else, S0, is susceptible to infection initially.
S0 = N - E0

# Parameters
β =  np.arange(0.1428, 1.5, 0.1)[0]
α = 2.0    #-set to quickly move S --> R1 at lockdown start
σ = np.arange(0.16, 0.5, 0.1)[0] # inverse of 2-6 days
ρ = np.arange(0.25, 0.5, 0.1)[0] # 25-50% of cases are asymptomatic
γA = np.arange(0.125, 0.33, 0.1)[0] # gammaA inverse of 4-14 days recovery
γM = np.arange(0.125, 0.33, 0.1)[0]
γH = np.arange(0.125, 0.33, 0.1)[0]
γC = np.arange(0.125, 0.33, 0.1)[0]
δ1 = np.arange(0.05, 1, 0.05)[0] #inverse of 1-10 days - modified to 1-20 days
δ2 = np.arange(0.06, 0.25, 0.05)[0] #inverse of 4-15 days
δ3 = np.arange(0.09, 1, 0.05)[0] #inverse of 1-11 days
m = np.arange(0.08, 0.25, 0.1)[0]
ξ = np.arange(0.1, 0.3, 0.1)[0] #proportion of symptomatic cases undetected
x1 = np.arange(0.05, 0.3, 0.1)[0]
x2 = np.arange(0.2, 0.3, 0.1)[0]
x3 = np.arange(0.2, 0.8, 0.1)[0]
d = 0.9 #np.arange(0.1, 0.4, 0.1)[0] #1-[0.58 0.85]
q = 0
λ = 4.6849

# A grid of time points (in days)
t = np.linspace(0, 365, 365)

# The SIR model differential equations.
def deriv(y, t, N, β, α, σ, ρ, γA, γM, γH, γC, δ1, δ2, δ3, m, ξ, x1, x2, x3, d, q, λ):
    S, E, IA, IP, IM, IH, IC, R1, R2, D = y
    dSdt = -d*β*S*(1-q)*(IA+IP+IM+IH+IC)/N
    dEdt = d*β*S*(1-q)*(IA+IP+IM+IH+IC)/N - σ*ρ*E - σ*(1-ρ)*E
    dIAdt = σ*ρ*E - γA*IA
    dIPdt = σ*(1-ρ)*E - (δ1*IP)
    dIMdt = δ1*IP - x1*δ2*IM - (1-x1)*γM*IM
    dIHdt = x1*δ2*IM - x2*δ3*IH - (1-x2)*γH*IH
    dICdt = x2*δ3*IH - (1-x3)*γC*IC - x3*m*IC
    dR1dt = 0
    dR2dt = γA*IA + (1-x1)*γM*IM + (1-x2)*γH*IH + (1-x3)*γC*IC
    dDdt  = x3*m*IC
    return dSdt, dEdt, dIAdt, dIPdt, dIMdt, dIHdt, dICdt, dR1dt, dR2dt, dDdt

# Initial conditions vector
y0 = S0, E0, IA0, IP0, IM0, IH0, IC0, R10, R20, D0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, β, α, σ, ρ, γA, γM, γH, γC, δ1, δ2, δ3, m, ξ, x1, x2, x3, d, q, λ))
S, E, IA, IP, IM, IH, IC, R1, R2, D = ret.T

df = pd.DataFrame()
df['t'] = t.astype(np.int64)
df['I'] = IA+IP+IM+IH+IC
df['H'] = IH + IC
df.to_csv ('no_variant.csv', index = False, header=True)
# Create traces
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.01, horizontal_spacing=0.05)
# fig.add_trace(go.Scatter(x=t, y=S, mode='lines', name='S', line = dict(color='blue')), row=1, col=1)
# fig.add_trace(go.Scatter(x=t, y=E, mode='lines', name='E', line = dict(color='orange')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=IA+IP+IM+IH+IC, mode='lines', name='Infections', line = dict(color='black')), row=1, col=1)
# # fig.add_trace(go.Scatter(x=t, y=IP, mode='lines', name='IP', line = dict(color='red')), row=1, col=1)
# # fig.add_trace(go.Scatter(x=t, y=IM, mode='lines', name='IM', line = dict(color='red', dash='dash')), row=1, col=1)
# # fig.add_trace(go.Scatter(x=t, y=IH, mode='lines', name='IH', line = dict(color='red', dash='dot')), row=1, col=1)
# # fig.add_trace(go.Scatter(x=t, y=IC, mode='lines', name='IC', line = dict(color='red', dash='dashdot')), row=1, col=1)
# # fig.add_trace(go.Scatter(x=t, y=R1, mode='lines', name='R1', line = dict(color='green', dash='dash')), row=1, col=1)
# fig.add_trace(go.Scatter(x=t, y=R2, mode='lines', name='R2', line = dict(color='green')), row=1, col=1)
# fig.add_trace(go.Scatter(x=t, y=D, mode='lines', name='D', line = dict(color='black')), row=1, col=1)

# I = All combined
# I Confirmed = ALl - IA (70% IP + IM + IH + IC)
# Hos = IH + IC



# fig.update_yaxes(type="log")
fig.update_layout(showlegend=True,  autosize=True, width=1024*1.5, height=800, xaxis = dict(dtick=10), yaxis = dict(dtick=N*0.1))
py.offline.plot(fig, filename='plot_temp.html')

'plot_temp.html'